In [1]:
import pandas as pd
import os

In [2]:
BASE_PATH = "/Users/beratzengin/Desktop/Github/EcoAir SmartCity Predictor/Data"
STATIONS_CSV = os.path.join(BASE_PATH, "stations_info.csv")

In [3]:
stations = pd.read_csv(STATIONS_CSV)

In [4]:
for index, row in stations.iterrows():
    # Sanitize station name for folder path
    station_name = row['Name'].replace(" ", "_").replace("/", "-")
    station_dir = os.path.join(BASE_PATH, station_name)
    
    if os.path.exists(station_dir):
        print(f"Merging Data for Station: {station_name}")
        
        for month in range(1, 13):
            month_str = f"{month:02d}"
            
            # File names based on your folder structure
            air_quality_file = os.path.join(station_dir, f"{month_str}_2024.csv")
            weather_file = os.path.join(station_dir, f"weather_{month_str}.csv")
            
            # Check if both files exist for the current month
            if os.path.exists(air_quality_file) and os.path.exists(weather_file):
                # Load datasets
                df_air = pd.read_csv(air_quality_file)
                df_weather = pd.read_csv(weather_file)
                
                # Ensure timestamps are in datetime format for accurate merging
                # Air quality uses 'date', Weather uses 'date'
                df_air['date'] = pd.to_datetime(df_air['date'], utc=True)
                df_weather['date'] = pd.to_datetime(df_weather['date'], utc=True)
                
                # Merge datasets on the 'date' column (Inner Join)
                # This ensures we only keep hours that exist in both files
                merged_df = pd.merge(df_air, df_weather, on='date', how='inner')
                
                # Save the merged result (e.g., merged_01_2024.csv)
                output_path = os.path.join(station_dir, f"merged_{month_str}_with_weather_2024.csv")
                merged_df.to_csv(output_path, index=False)
                
                print(f"Month {month_str} merged successfully.")
            else:
                if not os.path.exists(air_quality_file):
                    print(f"Missing Air Quality file for month {month_str}")
                if not os.path.exists(weather_file):
                    print(f"Missing Weather file for month {month_str}")
                    
    else:
        print(f"Station folder not found: {station_name}")

Merging Data for Station: Maslak


KeyError: 'date'

In [5]:
for index, row in stations.iterrows():
    station_name = row['Name'].replace(" ", "_").replace("/", "-")
    station_dir = os.path.join(BASE_PATH, station_name)
    
    if os.path.exists(station_dir):
        print(f"Processing & Parsing: {station_name}")
        
        for month in range(1, 13):
            month_str = f"{month:02d}"
            air_file = os.path.join(station_dir, f"{month_str}_2024.csv")
            weather_file = os.path.join(station_dir, f"weather_{month_str}.csv")
            
            if os.path.exists(air_file) and os.path.exists(weather_file):
                df_air = pd.read_csv(air_file)
                df_weather = pd.read_csv(weather_file)

                # 1. Parse JSON-like strings in 'Concentration' column
                # ast.literal_eval metni güvenli bir şekilde Python sözlüğüne çevirir
                df_air['Concentration'] = df_air['Concentration'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
                
                # Sözlük içindeki her bir anahtarı (PM10, SO2 vb.) yeni bir kolon yap
                concentration_cols = df_air['Concentration'].apply(pd.Series)
                df_air = pd.concat([df_air.drop('Concentration', axis=1), concentration_cols], axis=1)

                # 2. Standardize Date Columns for Merging
                # Air Quality has 'ReadTime', Weather has 'date'
                df_air = df_air.rename(columns={'ReadTime': 'date'})
                df_air['date'] = pd.to_datetime(df_air['date'], utc=True)
                df_weather['date'] = pd.to_datetime(pd.read_csv(weather_file)['date'], utc=True)

                # 3. Merge Datasets
                merged_df = pd.merge(df_air, df_weather, on='date', how='inner')
                
                # Drop unnecessary AQI column if needed, or keep it
                if 'AQI' in merged_df.columns:
                    merged_df = merged_df.drop(columns=['AQI'])

                if not merged_df.empty:
                    output_path = os.path.join(station_dir, f"merged_{month_str}_with_weather_2024.csv")
                    merged_df.to_csv(output_file, index=False)
                    print(f"Month {month_str}: Merged and JSON parsed.")
    else:
        print(f"Folder missing: {station_name}")

Processing & Parsing: Maslak


NameError: name 'ast' is not defined

In [6]:
import ast


In [7]:
for index, row in stations.iterrows():
    station_name = row['Name'].replace(" ", "_").replace("/", "-")
    station_dir = os.path.join(BASE_PATH, station_name)
    
    if os.path.exists(station_dir):
        print(f"Processing & Parsing: {station_name}")
        
        for month in range(1, 13):
            month_str = f"{month:02d}"
            air_file = os.path.join(station_dir, f"{month_str}_2024.csv")
            weather_file = os.path.join(station_dir, f"weather_{month_str}.csv")
            
            if os.path.exists(air_file) and os.path.exists(weather_file):
                df_air = pd.read_csv(air_file)
                df_weather = pd.read_csv(weather_file)

                # 1. Parse JSON-like strings in 'Concentration' column
                # ast.literal_eval metni güvenli bir şekilde Python sözlüğüne çevirir
                df_air['Concentration'] = df_air['Concentration'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
                
                # Sözlük içindeki her bir anahtarı (PM10, SO2 vb.) yeni bir kolon yap
                concentration_cols = df_air['Concentration'].apply(pd.Series)
                df_air = pd.concat([df_air.drop('Concentration', axis=1), concentration_cols], axis=1)

                # 2. Standardize Date Columns for Merging
                # Air Quality has 'ReadTime', Weather has 'date'
                df_air = df_air.rename(columns={'ReadTime': 'date'})
                df_air['date'] = pd.to_datetime(df_air['date'], utc=True)
                df_weather['date'] = pd.to_datetime(pd.read_csv(weather_file)['date'], utc=True)

                # 3. Merge Datasets
                merged_df = pd.merge(df_air, df_weather, on='date', how='inner')
                
                # Drop unnecessary AQI column if needed, or keep it
                if 'AQI' in merged_df.columns:
                    merged_df = merged_df.drop(columns=['AQI'])

                if not merged_df.empty:
                    output_path = os.path.join(station_dir, f"merged_{month_str}_with_weather_2024.csv")
                    merged_df.to_csv(output_file, index=False)
                    print(f"Month {month_str}: Merged and JSON parsed.")
    else:
        print(f"Folder missing: {station_name}")

Processing & Parsing: Maslak


NameError: name 'output_file' is not defined

In [8]:
import pandas as pd
import os
import ast  # Crucial for parsing JSON-like strings into dictionaries

# --- PATH CONFIGURATION ---
BASE_PATH = "/Users/beratzengin/Desktop/Github/EcoAir SmartCity Predictor/Data"
STATIONS_CSV = os.path.join(BASE_PATH, "stations_info.csv")

# Load station list
stations = pd.read_csv(STATIONS_CSV)

for index, row in stations.iterrows():
    # Folder name sanitization
    station_name = row['Name'].replace(" ", "_").replace("/", "-")
    station_dir = os.path.join(BASE_PATH, station_name)
    
    if os.path.exists(station_dir):
        print(f"🔗 Processing & Parsing: {station_name}")
        
        for month in range(1, 13):
            month_str = f"{month:02d}"
            air_file = os.path.join(station_dir, f"{month_str}_2024.csv")
            weather_file = os.path.join(station_dir, f"weather_{month_str}.csv")
            
            # Ensure both monthly files exist before merging
            if os.path.exists(air_file) and os.path.exists(weather_file):
                # Loading datasets
                df_air = pd.read_csv(air_file)
                df_weather = pd.read_csv(weather_file)

                # 1. Parse 'Concentration' column (Converts string dict to real Python dict)
                df_air['Concentration'] = df_air['Concentration'].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
                )
                
                # 2. Explode the dictionary into individual pollutant columns (PM10, NO2, etc.)
                concentration_cols = df_air['Concentration'].apply(pd.Series)
                df_air = pd.concat([df_air.drop('Concentration', axis=1), concentration_cols], axis=1)

                # 3. Synchronize timestamp columns for merging
                # Air Quality has 'ReadTime', Weather has 'date'
                df_air = df_air.rename(columns={'ReadTime': 'date'})
                df_air['date'] = pd.to_datetime(df_air['date'], utc=True)
                df_weather['date'] = pd.to_datetime(df_weather['date'], utc=True)

                # 4. Perform the Merge (Inner Join)
                merged_df = pd.merge(df_air, df_weather, on='date', how='inner')
                
                # Cleanup: Drop 'AQI' column if it exists to keep data lean
                if 'AQI' in merged_df.columns:
                    merged_df = merged_df.drop(columns=['AQI'])

                if not merged_df.empty:
                    output_path = os.path.join(station_dir, f"merged_{month_str}_2024.csv")
                    merged_df.to_csv(output_path, index=False)
                    print(f"Month {month_str}: Successfully merged and parsed.")
            else:
                pass # Skipping months with missing data
    else:
        print(f"Folder missing: {station_name}")

🔗 Processing & Parsing: Maslak
Month 01: Successfully merged and parsed.
Month 02: Successfully merged and parsed.
Month 03: Successfully merged and parsed.
Month 04: Successfully merged and parsed.
Month 05: Successfully merged and parsed.
Month 06: Successfully merged and parsed.
Month 07: Successfully merged and parsed.
Month 08: Successfully merged and parsed.
Month 09: Successfully merged and parsed.
Month 10: Successfully merged and parsed.
Month 11: Successfully merged and parsed.
Month 12: Successfully merged and parsed.
🔗 Processing & Parsing: Esenler
Month 01: Successfully merged and parsed.
Month 02: Successfully merged and parsed.
Month 03: Successfully merged and parsed.
Month 04: Successfully merged and parsed.
Month 05: Successfully merged and parsed.
Month 06: Successfully merged and parsed.
Month 07: Successfully merged and parsed.
Month 08: Successfully merged and parsed.
Month 09: Successfully merged and parsed.
Month 10: Successfully merged and parsed.
Month 11: Suc